# 🚖 IA - Homework 1 🚖
**👩 Alexandru Ioana  
 _📓 341C4_**
 
![Self driving cab](http://www.designmom.com/wp-content/uploads/2013/10/Google-Driverless-Taxi-Cab-537x389.jpg)

## Problem representation

The problem consists of **finding the optimal path of a cab which can pick up and drop off passengers on a given
map**. For a simple solution, we define helper classes with different functions.

### Coordinates

The `Coordinate` class offers a simple way of handling coordinates of problem variables (the car, passengers,
destinations).

In [1]:
class Coordinate(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return '<' + str(self.x) + ',' + str(self.y) + '>'

    def __eq__(self, other):
        if self is other:
            return True
        if not isinstance(other, Coordinate):
            return False
        return self.y == other.y and self.x == other.x

    def __repr__(self):
        return "Coordinate(%d, %d)" % (self.x, self.y)
    
    def __hash__(self):
        return hash(self.x) * 2 + hash(self.y) * 3

#### Manhattan distance

The **Manhattan distance** is the distance between two points measured along axes at right angles.

In order to calculate heuristics, we will need to calculate this distance between two coordinates.

![It's a surprise tool that will help us later](https://i.kym-cdn.com/photos/images/original/001/264/842/220.png)

In [2]:
def manhattan_distance(start, end):
    dx = abs(start.x - end.x)
    dy = abs(start.y - end.y)
    return dx + dy

### Actions

The `Action` enumeration defines all of the possible actions that the cab can take.

In [3]:
from enum import Enum

class Action(Enum):
    SOUTH = 'SOUTH'
    NORTH = 'NORTH'
    EAST = 'EAST'
    WEST = 'WEST'
    PICKUP = 'PICKUP'
    DROPOFF = 'DROPOFF'
    
    def __str__(self):
        return str(self.value)
    
    def __repr__(self):
        return str(self.value)

### Clients

The `Client` class represents the client in our problem. Each client has an initial (`start`) position on the map, a
`destination` they want to get to by taking a cab and a `budget` that they are willing to spend on the ride.

Two clients are the same (`__eq__`) if they share a destination, initial location and budget.

In [4]:
class Client(object):
    def __init__(self, start, destination, budget):
        self.start = start
        self.destination = destination
        self.budget = budget

    def __str__(self):
        return 'Client starts at ' + str(self.start) + \
               ', wants to get to ' + str(self.destination) + \
               ' and will spend at most ' + str(self.budget) + '.'

    def __eq__(self, other):
        if self is other:
            return True
        if not isinstance(other, Client):
            return False
        return self.start == other.start and\
               self.destination == other.destination and\
               self.budget == other.budget

    def __repr__(self):
        return "Client(%s, %s, %d)" % (str(self.start), str(self.destination), self.budget)
    
    def __hash__(self):
        return hash(self.start) * 2 + hash(self.destination) * 3 + hash(self.budget) * 5

### The cab

The `Cab` class represents a state of the single taxi cab in our problem. It has an initial `position` which can change
if it takes a movement action (`Action.NORTH`, `Action.SOUTH`, `Action.EAST`, `Action.WEST`) via the `do` method.
It also has an initial amount of `gas` (`max_gas`) which depletes by 1 unit with each movement.

The cab can hold either one or no `passenger` at a specific point in time. It can pick up a passenger when it is
located on the passenger's `start` position (`Action.PICKUP`), or drop off the passenger that is currently riding
(`Action.DROPOFF`) when it is located on the passenger's wanted `destination`, at which point the cab's `balance` will
increase by the passenger's `budget`.

Based on the map information, when in a specific position the cab can only take a specific set of actions, which can be
determined via the `get_available_actions` method.

Two different states of the cab are the same (`__eq__`) if they share the position, the passenger and the balance. The
actions taken as well as the amount of gas remaining are not taken into account in order to avoid running in circles.

In [5]:
from random import shuffle

class Cab(object):
    def __init__(self, position, gas):
        self.position = position
        self.passenger = None
        self.gas = gas
        self.max_gas = gas
        self.balance = 0
        self.actions = []
        
    def __str__(self):
        has_passenger = 'has a passenger' if self.passenger is not None else 'does not have a passenger'
        return 'Cab is at position ' + str(self.position) +\
               ', has ' + str(self.gas) + '/' + str(self.max_gas) +' gas, ' +\
               str(self.balance)+ '$ and ' + has_passenger + '.'

    def __eq__(self, other):
        if self is other:
            return True
        if not isinstance(other, Cab):
            return False
        return self.position == other.position and\
               self.passenger == other.passenger and\
               self.balance == other.balance

    def __repr__(self):
        return "Cab(%s)" % str(self.position)
    
    def __hash__(self):
        return hash(self.position) * 2 + hash(self.passenger is None) * 3
        
    ### Check if actions are possible
    
    def out_of_gas(self):
        return self.gas  == 0
            
    def get_available_actions(self, world):
        actions = []
        if not self.out_of_gas():
            if self.__can_dropoff(world):
                actions.append(Action.DROPOFF)
            if self.__can_pickup(world):
                actions.append(Action.PICKUP)
            if self.__can_go_east(world):
                actions.append(Action.EAST)
            if self.__can_go_west(world):
                actions.append(Action.WEST)
            if self.__can_go_south(world):
                actions.append(Action.SOUTH)
            if self.__can_go_north(world):
                actions.append(Action.NORTH)
        shuffle(actions)
        return actions
        
    def __can_go_west(self, world):
        grid_coords = Coordinate(self.position.x + 1, self.position.y * 2 + 1)
        return self.position.y > 0 and world.grid[grid_coords.x][grid_coords.y - 1] != '|'
        
    def __can_go_east(self, world):
        grid_coords = Coordinate(self.position.x + 1, self.position.y * 2 + 1)
        return self.position.y < world.width and world.grid[grid_coords.x][grid_coords.y + 1] != '|'
        
    def __can_go_north(self, world):
        return self.position.x > 0
        
    def __can_go_south(self, world):
        return self.position.x < world.height - 1
    
    def __can_pickup(self, world):
        return self.passenger is None and list(filter(lambda x : x.start == self.position, world.clients))
        
    def __can_dropoff(self, world):
        return self.passenger is not None and self.passenger.destination == self.position
    
    ### Do actions
    
    def do(self, action, world):
        self.actions.append(action)
        if action == Action.EAST:
            self.__go_east(world)
        elif action== Action.WEST:
            self.__go_west(world)
        elif action== Action.SOUTH:
            self.__go_south(world)
        elif action== Action.NORTH:
            self.__go_north(world)
        elif action== Action.PICKUP:
            self.__pickup(world)
        elif action== Action.DROPOFF:
            self.__dropoff(world)
    
    def __go_east(self, world):
        assert not self.out_of_gas() and self.__can_go_east(world)
        self.gas -= 1
        self.position.y += 1
    
    def __go_west(self, world):
        assert not self.out_of_gas() and self.__can_go_west(world)
        self.gas -= 1
        self.position.y -= 1
        
    def __go_south(self, world):
        assert not self.out_of_gas() and self.__can_go_south(world)
        self.gas -= 1
        self.position.x += 1
        
    def __go_north(self, world):
        assert not self.out_of_gas() and self.__can_go_north(world)
        self.gas -= 1
        self.position.x -= 1
        
    def __pickup(self, world):
        assert not self.out_of_gas() and self.__can_pickup(world)
        client = list(filter(lambda x : x.start == self.position, world.clients))[0]
        self.passenger = client
        world.clients.remove(client)
    
    def __dropoff(self, world):
        assert not self.out_of_gas() and self.__can_dropoff(world)
        self.balance += self.passenger.budget
        self.passenger = None
        

### The world

We are representing the problem as a state ***space search problem***. A specific state in our problem consists of the
map (`grid`), awaiting `clients` on the map, as well as a single `cab`. We will call this state our `world`.

Since it is a space search problem, it has the following properties:
- any given state can be **final** (we can check whether a state is final by using the `goal()` method);
- from any given state, we can **transition** into a number of other states (which we can get by calling the transition
  function, `succ()`);
- a state has a specific **cost** associated with it, representing the amount of gas spent from the initial state up to
  the given state, which we can obtain by calling `g()`.

We consider two separate states of the problem to be the same when the state of the cab is the same (see
[The cab](#The-cab)) and the list of awaiting clients is the same.

In [6]:
# coding=utf-8
from copy import deepcopy, copy

use_emojis = True
class World(object):
    def __init__(self, grid, cab, clients):
        self.width = len(grid[0]) // 2
        self.height = len(grid) - 2
        self.grid = grid
        self.cab = cab
        self.clients = clients

    def __str__(self):
        string = ''
        for i, line in enumerate(self.grid, start=0):
            for j, cell in enumerate(line, start=0):
                coords = Coordinate(i - 1, j // 2)
                if j % 2 == 0 and list(filter(lambda x: x.start == coords, self.clients)):
                    string += '🚶' if use_emojis else 'C'
                elif j % 2 == 0 and self.cab.passenger is not None and self.cab.passenger.destination == coords:
                    string += '🚏' if use_emojis else 'D'
                elif j % 2 == 1 and self.cab.position == coords:
                    string += '🚘️' if use_emojis else 'T'
                elif i == j == 0:
                    string += '┏'
                elif i == 0 and j == len(line) - 1:
                    string += '┓'
                elif i == len(self.grid) - 1 and j == 0:
                    string += '┗'
                elif i == len(self.grid) - 1 and j == len(line) - 1:
                    string += '┛'
                elif cell == '|':
                    string += '┃'
                elif cell == ':':
                    string += '┆'
                elif cell == '-':
                    if i > 0 and self.grid[i - 1][j] != ' ':
                        string += '┻'
                    elif i < len(self.grid) - 1 and self.grid[i + 1][j] != ' ':
                        string += '┳'
                    else:
                        string += '━'
                else:
                    string += cell
            string += '\n'
        return string

    def __repr__(self):
        return str(self)
    
    def __eq__(self, other):
        if self is other:
            return True
        if not isinstance(other, World):
            return False
        return self.cab == other.cab and set(self.clients) == set(other.clients)
    
    def __hash__(self):
        return hash(self.cab)
    
    def goal(self):
        return self.cab.passenger is None and\
                (len(self.clients) == 0 or\
                 self.cab.out_of_gas())
    
    def succ(self):
        states = []
        actions = self.cab.get_available_actions(self)
        for action in actions:
            cab_copy = deepcopy(self.cab)
            clients_copy = copy(self.clients)
            next = World(self.grid, cab_copy, clients_copy)
            cab_copy.do(action, next)
            states.append(next)
        return states
    
    def g(self):
        return self.cab.max_gas - self.cab.gas
    
    def h(self):
        return 0

## I/O operations

In addition to implementing the `__str__` and `__repr__` methods for all of our classes, we will define functions for
reading and writing of the problem data.

### Reading

The `read_data` function opens the file with the given `filename` and turns the data inside it into an instance of
`World`.

In [7]:
def read_data(filename):
    with open(filename, 'r') as f:
        height, width, gas = [int(x) for x in next(f).split()]
        car_position_y, car_position_x = [int(x) for x in next(f).split()]
        cab = Cab(Coordinate(car_position_x, car_position_y), gas)
        
        client_count = int(next(f))
        clients = []
        for i in range(client_count):
            start_y, start_x, destination_y, destination_x, budget = [int(x) for x in next(f).split()]
            clients.append(Client(Coordinate(start_x, start_y), Coordinate(destination_x, destination_y), budget))

        grid = []
        for i in range(height + 2):
            grid.append(list(next(f).rstrip()))
            
        return World(grid, cab, clients)

### Writing

The solution of our problem consists of a final state and a dictionary of parents. We have multiple ways of displaying
that result.

#### Listing

The `print_result` function lists each step defined in a solution, from the initial state up to the final one. 

In [8]:
def print_result(state, parents, closed=None):
    next = parents[state]
    if next is not None:
        print_result(next, parents)
    print(state)

#### Animating

The `animate_result` function works similarly with `print_result`, except it only shows each frame (step) for 0.4
seconds before clearing the display and showing the following frame.

In [9]:
from IPython.core.display import clear_output
import time

def animate_result(state, parents, closed=None):
    next = parents[state]
    if next is not None:
        print_result(next, parents)
    string = str(state)
    clear_output()
    print(string)
    time.sleep(0.4)

## Search algorithms

We will solve the problem using different search algorithms (both informed and uninformed) in order to compare their
performance, caveats and particularities.

While we can represent the solution in an intuitive way by drawing the map at each step (see [Writing](#Writing)), that
can quickly become impractical for more complex inputs. A simpler way of representing a discovered solution is with a
list of actions in the order that they are to be taken by the cab, as well as the total cost of that set of actions.
That is what the `get_solution` function is doing.

In [10]:
def get_solution(state, parents=None, closed=None):
    actions = state.cab.actions
    cost = state.g()
    return actions, cost

Additionally, not all solutions are equivalent. We can evaluate the quality of a solution by adding the remaining fuel and the final balance:

In [11]:
def evaluate_solution(state, parents=None, closed=None):
    return state.cab.gas + state.cab.balance

In [12]:
def visited_states(state, parents, closed):
    return len(closed)

### Uninformed search strategies

An **uninformed search** is a searching technique that has _no additional information about the distance from the
current state to the goal_.

#### Breadth-first search

**Breadth-first search (BFS)** is an algorithm which starts at the initial state (_s0_) and explores all of the possible
states that can be transitioned to at the present depth prior to moving on to the states at the next depth level. 

A naïve implementation of BFS simply adds all states that can be explored to the back of a queue (we will also make sure
not to have duplicates in the queue, since it would be unbearably slow otherwise):

In [13]:
def bfs(world):
    open = [world]
    parents = {world : None}
    while len(open) > 0:
        current = open.pop(0)
        if current.goal():
            return current, parents
        for next in current.succ():
            if next not in open:
                parents[next] = current
                open.append(next)
    return None

A simple way of optimizing this algorithm is by saving the visited nodes and not processing them again:

In [14]:
def optimized_bfs(world):
    open = [world]
    closed = []
    parents = {world : None}
    while len(open) > 0:
        current = open.pop(0)
        if current.goal():
            return current, parents
        for next in current.succ():
            if next not in open and next not in closed:
                parents[next] = current
                open.append(next)
        closed.append(current)
    return None

Let's check that both algorithms lead to the same solution:

In [15]:
world = read_data('input')
bfs1 = get_solution(*bfs(world))
bfs2 = get_solution(*optimized_bfs(world))
assert bfs1 == bfs2
print(bfs1)

([NORTH, WEST, SOUTH, SOUTH, PICKUP, NORTH, NORTH, NORTH, NORTH, DROPOFF], 8)


Now let's compare their performance:

In [16]:
%timeit bfs(world)
%timeit optimized_bfs(world)

10 loops, best of 3: 40.9 ms per loop
100 loops, best of 3: 12.5 ms per loop


As we can see, the optimized version is significantly faster. That is due to the fact that the naïve version is likely
to check the same state multiple times, hence wasting time without gaining any additional information.

#### Uniform cost search

**Uniform Cost Search (UCS)** is similar to **BFS**, except instead of exploring each state in the order it was
discovered, it prioritizes exploring the states which have a lower cost associated with them (in our case, a lower
amount of gas used by the cab in order to arrive to that state, or `g()`) by using a priority queue.

In [17]:
# Using heapq to implement the priority queue
import heapq

In [18]:
# Define a sub-class which orders instances based on g()
class UniformWorld(World):
    def __lt__(self, other):
        return self.g() < other.g()

As with **BFS**, we can have the naïve implementation:

In [19]:
def ucs(world):
    open = [world]
    parents = {world : None}
    while len(open) > 0:
        current = open.pop(0)
        if current.goal():
            return current, parents
        for next in current.succ():
            if next not in open:
                parents[next] = current
                # heapq uses the __lt__ method to sort the states
                next.__class__ = UniformWorld
                heapq.heappush(open, next)
    return None

And the optimized one:

In [20]:
def optimized_ucs(world):
    open = [world]
    closed = []
    parents = {world : None}
    while len(open) > 0:
        current = open.pop(0)
        if current.goal():
            return current, parents, closed
        for next in current.succ():
            if next not in open and next not in closed:
                parents[next] = current
                # heapq uses the __lt__ method to sort the states
                next.__class__ = UniformWorld
                heapq.heappush(open, next)
        closed.append(current)
    return None

As before, both versions should lead to the same solution:

In [21]:
world = read_data('input')
ucs1 = get_solution(*ucs(world))
ucs2 = get_solution(*optimized_ucs(world))
assert ucs1 == ucs2
print(ucs1)

([NORTH, WEST, SOUTH, SOUTH, PICKUP, NORTH, NORTH, NORTH, NORTH, DROPOFF], 8)


Now let's compare their performance:

In [22]:
%timeit ucs(world)
%timeit optimized_ucs(world)

10 loops, best of 3: 26.5 ms per loop
100 loops, best of 3: 10.3 ms per loop


As expected, the optimized algorithm is significantly faster.

#### Depth-first search

**Depth-first search (DFS)** is different from **BFS** in that it explores as far (deep) as possible along each path
before backtracking.

This behaviour can be obtained by using a stack instead of a queue (or in our case, inserting elements at the beginning
of the list):

In [23]:
def dfs(world):
    open = [world]
    parents = {world : None}
    while len(open) > 0:
        current = open.pop(0)
        if current.goal():
            return current, parents
        for next in current.succ():
            if next not in open:
                parents[next] = current
                open.insert(0, next)
    return None

We can once again optimize by saving the visited states:

In [24]:
def optimized_dfs(world):
    open = [world]
    closed = []
    parents = {world : None}
    while len(open) > 0:
        current = open.pop(0)
        if current.goal():
            return current, parents, closed
        for next in current.succ():
            if next not in open and next not in closed:
                parents[next] = current
                open.insert(0, next)
        closed.append(current)
    return None

This time, the naïve implementation's solution will most likely just be running around until out of gas, while the
optimized solution will offer a solution which might not always be the most cost-efficient one. This behaviour is due to
the fact that **DFS** will return the first final state it reaches, which will usually not be the most cost-effective
one because `succ` returns the following states in a random order.

In [27]:
world = read_data('input')
dfs1 = get_solution(*dfs(world))
dfs2 = get_solution(*optimized_dfs(world))
print(dfs1)
print(dfs2)

([SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH], 100)
([NORTH, WEST, SOUTH, SOUTH, PICKUP, NORTH, NORTH, EAST, NORTH, NORTH, WEST, DROPOFF], 10)


The performance difference is even more obvious here:

In [29]:
%timeit dfs(world)
%timeit optimized_dfs(world)

10 loops, best of 3: 26.2 ms per loop
100 loops, best of 3: 11.5 ms per loop


#### Depth-limited search

**Depth-limited search (DLS)** improves upon **DFS** by that it limits the maximum depth that the algorithm can get to,
in order to avoid recursing infinitely in case the goal is unreachable or going too far onto the wrong path.

The caveat that this version adds is the fact that, if the cutoff is too low, we may encounter a "false" failure caused
by the goal not being reachable within the set depth.

> ***NOTE:*** Our problem is technically already depth-limited by design, since the cab cannot move more than its gas
> tank allows it to move. It could still make a virtually unlimited amount of PICKUP/DROPOFF actions, but since the
> number of clients on the map is also finite, that is not possible either.

The naïve solution:

In [30]:
def dls(world, k):
    open = [(0, world)]
    parents = {world : None}
    while len(open) > 0:
        depth, current = open.pop(0)
        if current.goal():
            return current, parents
        if depth >= k:
            continue
        for next in current.succ():
            if next not in [x[1] for x in open]:
                parents[next] = current
                open.insert(0, (depth + 1, next))
    return None

The optimized solution:

In [31]:
def optimized_dls(world, k):
    open = [(0, world)]
    closed = []
    parents = {world : None}
    while len(open) > 0:
        depth, current = open.pop(0)
        if current.goal():
            return current, parents, closed
        if depth >= k:
            continue
        for next in current.succ():
            if (next not in [x[1] for x in open]) and (next not in closed):
                parents[next] = current
                open.insert(0, (depth + 1, next))
        closed.append(current)
    return None

Four our sample data, we already know that the ideal sequence of actions has a length of 10 and a cost of 8, therefore
`k = 10` should be deep enough for us to find this solution:

In [32]:
world = read_data('input')
dls1 = get_solution(*dls(world, 10))
dls2 = get_solution(*optimized_dls(world, 10))
assert dls1 == dls2
print(dls1)

([NORTH, WEST, SOUTH, SOUTH, PICKUP, NORTH, NORTH, NORTH, NORTH, DROPOFF], 8)


As we can see, this time both the naïve and the optimized versions eventually reach the same (good) solution thanks to
the cutoff value, which prevents them from going in the wrong direction.

Now for the performance:

In [33]:
%timeit dls(world, 10)
%timeit optimized_dls(world, 10)

The slowest run took 27.25 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 21 ms per loop
100 loops, best of 3: 6.69 ms per loop


Since the naïve solution is likely to go off track many times before finally finding the good solution, it is incredibly
slow compared to the optimized one.

#### Iterative deepening search

**Iterative deepening search** takes **DLS** one step further by incrementally trying different values for the depth
until a solution is found.

We can use the simple **DLS** algorithm:

In [34]:
def ids(world):
    k = 0
    while True:
        result = dls(world, k)
        if result is not None:
            return result
        k += 1     

Or the optimized one:

In [35]:
def optimized_ids(world):
    k = 0
    while True:
        result = optimized_dls(world, k)
        if result is not None:
            return result
        k += 1  

We _cheated_ earlier since we already knew a good value for `k`. However, most of the time we won't know the value
beforehand, so the best way to find the minimum value of `k` as well as the best solution is to increment `k` and keep
trying **DLS** until we succeed. We already know that we'll reach a good solution here when k becomes 10:

In [36]:
world = read_data('input')
ids1 = get_solution(*ids(world))
ids2 = get_solution(*optimized_ids(world))
assert ids1 == ids2
print(ids1)

([NORTH, WEST, SOUTH, SOUTH, PICKUP, NORTH, NORTH, NORTH, NORTH, DROPOFF], 8)


As for the performance, it is slow compared to the other algorithms since it involves running **DLS** multiple times:

In [37]:
%timeit ids(world)
%timeit optimized_ids(world)

1 loop, best of 3: 1.75 s per loop
10 loops, best of 3: 48.3 ms per loop


### Informed search strategies

**Informed search strategies** use additional information about states (heuristics - `h()` - which estimate how far from
the goal a given state is) that have not yet been explored to decide which state to examine next.

One possible heuristic for our problem is the Manhattan distance from the current position of the cab to either the
clients's destination (if the cab has a passenger), or the closest potential client:

In [38]:
def h1(world):
    if world.goal():
        return 0
    if world.cab.passenger is not None:  # distance to the passenger's destination
        return manhattan_distance(world.cab.position, world.cab.passenger.destination)
    else:  # distance to the closest potential client
        return min(manhattan_distance(world.cab.position, client.start) for client in world.clients)

Another heuristic could also take into account the client's budget in addition to the distance (the higher the budget,
the lower the impact a longer distance will have on the cost):

In [39]:
def h2(world):
    if world.goal():
        return 0
    if world.cab.passenger is not None:  # distance to the passenger's destination
        return manhattan_distance(world.cab.position, world.cab.passenger.destination)
    else:  # distance to the closest potential client divided by their budget
        return min(manhattan_distance(world.cab.position, client.start) / client.budget for client in world.clients)

#### Greedy best-first search

The **Greedy best-first search** algorithm is similar to **UCS**, except that it uses heuristics. Instead of selecting
the state closest to the starting point, it selects the state closest to the goal. **GBFS** is not guaranteed to find
the most cost-efficient path. However, it runs much quicker than **UCS** because it uses the heuristic function to guide
its way towards the goal very quickly.

In [40]:
# Define a sub-class which orders instances based on h()
class GreedyWorld(World):
    def __lt__(self, other):
        return self.h() < other.h()

Since it is similar to **UCS**, it also has a naïve solution:

In [41]:
def gbfs(world, heuristic):
    World.h = heuristic
    open = [world]
    parents = {world : None}
    while len(open) > 0:
        current = open.pop(0)
        if current.goal():
            return current, parents
        for next in current.succ():
            if next not in open:
                parents[next] = current
                # heapq uses the __lt__ method to sort the states
                next.__class__ = GreedyWorld
                heapq.heappush(open, next)
    return None

As well as an optimized one:

In [42]:
def optimized_gbfs(world, heuristic):
    World.h = heuristic
    open = [world]
    closed = []
    parents = {world : None}
    while len(open) > 0:
        current = open.pop(0)
        if current.goal():
            return current, parents, closed
        for next in current.succ():
            if next not in open and next not in closed:
                parents[next] = current
                # heapq uses the __lt__ method to sort the states
                next.__class__ = GreedyWorld
                heapq.heappush(open, next)
        closed.append(current)
    return None

On our sample data, like the unoptimized **DFS**, the unoptimized **GBFS** goes into a loop since, because it only takes
into account the Manhattan distance between a state and the destination (be it the closest client or the passenger's
destination) and there is an obstacle between the initial state and the closest client, making it keep moving between
two states which don't allow it to reach the client.

The optimized version, however, quickly reaches the solution:

In [43]:
world = read_data('input')
gbfs1 = get_solution(*gbfs(world, h1))
gbfs2 = get_solution(*optimized_gbfs(world, h1))
print(gbfs1)
print(gbfs2)

([SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH, SOUTH, NORTH], 100)
([NORTH, WEST, SOUTH, SOUTH, PICKUP, NORTH, NORTH, NORTH, NORTH, DROPOFF], 8)


The performance is very good:

In [44]:
%timeit gbfs(world, h1)
%timeit optimized_gbfs(world, h1)

10 loops, best of 3: 45.6 ms per loop
100 loops, best of 3: 4.23 ms per loop


#### A* Search

A* combines heuristic approaches like **GBFS** and formal approaches like **UCS**. Unlike the majority of heuristic
approaches, which usually give you an approximate way to solve problems without guaranteeing that you get the best
answer, A* can guarantee a shortest path, even though the heuristic itself does not give any guarantees.

In [45]:
# Define a sub-class which orders instances based on both g() and h()
class SmartWorld(World):
    def __lt__(self, other):
        return self.g() + self.h() < other.g() + other.h()

The unoptimized A* is:

In [46]:
def a_star(world, heuristic):
    World.h = heuristic
    open = [world]
    parents = {world : None}
    while len(open) > 0:
        current = open.pop(0)
        if current.goal():
            return current, parents
        for next in current.succ():
            if next not in open:
                parents[next] = current
                # heapq uses the __lt__ method to sort the states
                next.__class__ = SmartWorld
                heapq.heappush(open, next)
    return None

And the optimized version is:

In [47]:
def optimized_a_star(world, heuristic):
    World.h = heuristic
    open = [world]
    closed = []
    parents = {world : None}
    while len(open) > 0:
        current = open.pop(0)
        if current.goal():
            return current, parents, closed
        for next in current.succ():
            if next not in open and next not in closed:
                parents[next] = current
                # heapq uses the __lt__ method to sort the states
                next.__class__ = SmartWorld
                heapq.heappush(open, next)
        closed.append(current)
    return None

Since the cost takes into account both distance and gas cost, both versions will reach a good solution:

In [48]:
world = read_data('input')
a_star1 = get_solution(*a_star(world, h1))
a_star2 = get_solution(*optimized_a_star(world, h1))
print(a_star1)
print(a_star2)

([NORTH, WEST, SOUTH, SOUTH, PICKUP, NORTH, NORTH, NORTH, NORTH, DROPOFF], 8)
([NORTH, WEST, SOUTH, SOUTH, PICKUP, NORTH, NORTH, NORTH, NORTH, DROPOFF], 8)


The optimized algorithm is very fast, but the unoptimized one is not too bad either:

In [49]:
%timeit a_star(world, h1)
%timeit optimized_a_star(world, h1)

The slowest run took 6.72 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 15.3 ms per loop
100 loops, best of 3: 6.28 ms per loop


#### Hill-Climbing Search

In [50]:
def hcs(world, heuristic):
    World.h = heuristic
    current = world
    parents = {world : None}
    done = False
    current_cost = current.g() + current.h()
    while not done:
        maximum = current_cost
        next_state = None
        for next in current.succ():
            cost = next.g() + next.h()
            if cost > maximum:
                maximum = cost
                next_state = next
        if next_state is None:
            done = True
        else:
            parents[next_state] = current
            current = next_state
            current_cost = maximum
    return current, parents

In [51]:
def optimized_hcs(world, heuristic):
    World.h = heuristic
    current = world
    parents = {world : None}
    done = False
    current_cost = current.g() + current.h()
    closed = []
    while not done:
        maximum = current_cost
        next_state = None
        for next in current.succ():
            if next in closed:
                continue
            cost = next.g() + next.h()
            if cost > maximum:
                maximum = cost
                next_state = next
        closed.append(current)
        if next_state is None:
            done = True
        else:
            parents[next_state] = current
            current = next_state
            current_cost = maximum
    return current, parents, closed

In [52]:
world = read_data('input')
hcs1 = get_solution(*hcs(world, h1))
hcs2 = get_solution(*optimized_hcs(world, h1))
print(hcs1)
print(hcs2)

([NORTH, EAST, NORTH, NORTH, EAST, EAST], 6)
([EAST, NORTH, NORTH, EAST, NORTH, EAST], 6)


In [53]:
%timeit hcs(world, h1)
%timeit optimized_hcs(world, h1)

The slowest run took 12.54 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.41 ms per loop
1000 loops, best of 3: 1.53 ms per loop


## Comparative analysis

We will be analyzing the above strategies in terms of performance (based on the running time), quality of results (using `evaulate_solution`) and number of explored states (`visited_states`) on the data in the `tests` folder.

In [54]:
times = {}
quality = {}
visited = {}

### Data

The initial states we will be analyzing are the following:

In [55]:
data = [read_data(filename) for filename in ['tests/test' + str(x) + '.in' for x in range(1, 7)]]
use_emojis = False
for i, d in enumerate(data):
    print(str(i) + ') ' + str(d.cab))
    print(d)

0) Cab is at position <2,2>, has 5/5 gas, 0$ and does not have a passenger.
┏━┳━┳━┳━┳━┓
┃ ┆ ┃ ┆ ┆ ┃
C C ┆ ┆ ┆ ┃
┃ ┆ CT┆ ┆ ┃
┃ ┃ ┆ ┃ ┆ ┃
┃ ┃ ┆ ┃ ┆ ┃
┗━┻━┻━┻━┻━┛

1) Cab is at position <1,2>, has 15/15 gas, 0$ and does not have a passenger.
┏━┳━┳━┳━┳━┓
┃ C C ┆ C ┃
C C ┆T┆ C ┃
C C C ┆ ┆ ┃
┃ ┃ ┆ C ┆ ┃
┃ ┃ ┆ ┃ ┆ ┃
┗━┻━┻━┻━┻━┛

2) Cab is at position <1,2>, has 20/20 gas, 0$ and does not have a passenger.
┏━┳━┳━┳━┳━┓
┃ C C ┆ C ┃
C C ┆T┆ C ┃
C C C ┆ ┆ ┃
┃ ┃ ┆ C ┆ ┃
┃ ┃ ┆ ┃ ┆ ┃
┗━┻━┻━┻━┻━┛

3) Cab is at position <5,1>, has 20/20 gas, 0$ and does not have a passenger.
┏━┳━┳━┳━┳━┳━┳━┓
┃ C C ┆ ┆ ┃ ┆ ┃
┃ ┆ ┆ ┆ ┆ C ┆ ┃
┃ C ┆ ┆ ┆ ┆ ┆ ┃
┃ ┃ ┆ ┃ ┆ ┆ ┃ ┃
┃ ┃ C ┃ ┆ ┆ ┆ ┃
┃ CT┆ ┆ ┆ ┆ ┆ ┃
C ┆ ┆ ┃ ┆ ┃ ┆ ┃
┗━┻━┻━┻━┻━┻━┻━┛

4) Cab is at position <5,1>, has 25/25 gas, 0$ and does not have a passenger.
┏━┳━┳━┳━┳━┳━┳━┓
┃ C C ┆ ┆ ┃ ┆ ┃
┃ ┆ ┆ ┆ ┆ C ┆ ┃
┃ C ┆ ┆ ┆ ┆ ┆ ┃
┃ ┃ C ┃ ┆ ┆ ┃ ┃
┃ ┃ C ┃ ┆ ┆ ┆ ┃
┃ CT┆ ┆ ┆ ┆ ┆ ┃
C ┆ ┆ ┃ ┆ ┃ ┆ ┃
┗━┻━┻━┻━┻━┻━┻━┛

5) Cab is at position <8,3>, has 25/25 gas, 0$ and does not have a 

### Performance

Let's run all of the strategies on all of the inputs several times and calculate the average time:

In [ ]:
times['BFS'] = []
for d in data:
    timeitresult = %timeit -o -q optimized_bfs(d)
    times['BFS'].append(sum(timeitresult.all_runs) / len(timeitresult.all_runs))

In [ ]:
times['DFS'] = []
for d in data:
    timeitresult = %timeit -o -q optimized_dfs(d)
    times['DFS'].append(sum(timeitresult.all_runs) / len(timeitresult.all_runs))

In [ ]:
times['UCS'] = []
for d in data:
    timeitresult = %timeit -o -q optimized_ucs(d)
    times['UCS'].append(sum(timeitresult.all_runs) / len(timeitresult.all_runs))

In [ ]:
times['IDS'] = []
for d in data:
    timeitresult = %timeit -o -q optimized_ids(d)
    times['IDS'].append(sum(timeitresult.all_runs) / len(timeitresult.all_runs))

In [ ]:
times['GBFS1'] = []
for d in data:
    timeitresult = %timeit -o -q optimized_gbfs(d, h1)
    times['GBFS1'].append(sum(timeitresult.all_runs) / len(timeitresult.all_runs))

In [ ]:
times['GBFS2'] = []
for d in data:
    timeitresult = %timeit -o -q optimized_gbfs(d, h2)
    times['GBFS2'].append(sum(timeitresult.all_runs) / len(timeitresult.all_runs))

In [ ]:
times['A*1'] = []
for d in data:
    timeitresult = %timeit -o -q optimized_a_star(d, h1)
    times['A*1'].append(sum(timeitresult.all_runs) / len(timeitresult.all_runs))

In [ ]:
times['A*2'] = []
for d in data:
    timeitresult = %timeit -o -q optimized_a_star(d, h2)
    times['A*2'].append(sum(timeitresult.all_runs) / len(timeitresult.all_runs))

In [ ]:
times['HCS1'] = []
for d in data:
    timeitresult = %timeit -o -q optimized_hcs(d, h1)
    times['HCS1'].append(sum(timeitresult.all_runs) / len(timeitresult.all_runs))

In [ ]:
times['HCS2'] = []
for d in data:
    timeitresult = %timeit -o -q optimized_hcs(d, h2)
    times['HCS2'].append(sum(timeitresult.all_runs) / len(timeitresult.all_runs))

Now to plot the data we just gathered:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
def plot_algorithms(data, label,xlim=[0, 5], ylim=None):
    algorithms = ['BFS', 'DFS', 'UCS', 'IDS']
    plt.figure(figsize=(15,15))
    for a in algorithms:
        plt.plot(a, data=data, label=a)
    colors = ['C4', 'C5', 'C6']
    algorithms = ['GBFS1', 'A*1', 'HCS1']
    for a, c in zip(algorithms, colors):
        plt.plot(a, data=data, label=a, linestyle='dotted', color=c)
    algorithms = ['GBFS2', 'A*2', 'HCS2']
    for a, c in zip(algorithms, colors):
        plt.plot(a, data=data, label=a, linestyle='dashed', color=c)
    plt.xlabel('input')
    plt.ylabel(label)
    plt.xlim(xlim)
    max_y = max(list([val for values in data.values() for val in values])) + 1
    plt.ylim([0, max_y])
    y_range = range(0, int(max_y + 10), 10)
    plt.yticks(y_range, y_range)

    plt.legend()
    plt.show()

In [ ]:
%matplotlib inline
plot_algorithms(times, 'time(s)')

### Quality

Speed is not the only relevant factor. As we've seen before, different algorithms will yield different results. We will check the quality of these results by calling `evaluate_solution`, which sums up the remaining gas and the money.

In [ ]:
quality['BFS'] = []
for d in data:
    quality['BFS'].append(evaluate_solution(*optimized_bfs(d)))

In [ ]:
quality['DFS'] = []
for d in data:
    quality['DFS'].append(evaluate_solution(*optimized_dfs(d)))

In [ ]:
quality['UCS'] = []
for d in data:
    quality['UCS'].append(evaluate_solution(*optimized_ucs(d)))

In [ ]:
quality['IDS'] = []
for d in data:
    quality['IDS'].append(evaluate_solution(*optimized_ids(d)))

In [ ]:
quality['GBFS1'] = []
for d in data:
    quality['GBFS1'].append(evaluate_solution(*optimized_gbfs(d, h1)))

In [ ]:
quality['GBFS2'] = []
for d in data:
    quality['GBFS2'].append(evaluate_solution(*optimized_gbfs(d, h2)))

In [ ]:
quality['A*1'] = []
for d in data:
    quality['A*1'].append(evaluate_solution(*optimized_a_star(d, h1)))

In [ ]:
quality['A*2'] = []
for d in data:
    quality['A*2'].append(evaluate_solution(*optimized_a_star(d, h2)))

In [ ]:
quality['HCS1'] = []
for d in data:
    quality['HCS1'].append(evaluate_solution(*optimized_hcs(d, h1)))

In [ ]:
quality['HCS2'] = []
for d in data:
    quality['HCS2'].append(evaluate_solution(*optimized_hcs(d, h2)))

In [ ]:
%matplotlib inline
plot_algorithms(quality, 'quality(gas+money)')

### States

Speed should be directly related to the number of visited states.

In [ ]:
visited['BFS'] = []
for d in data:
    visited['BFS'].append(visited_states(*optimized_bfs(d)))

In [ ]:
visited['DFS'] = []
for d in data:
    visited['DFS'].append(visited_states(*optimized_dfs(d)))

In [ ]:
visited['UCS'] = []
for d in data:
    visited['UCS'].append(visited_states(*optimized_ucs(d)))

In [ ]:
visited['IDS'] = []
for d in data:
    visited['IDS'].append(visited_states(*optimized_ids(d)))

In [ ]:
visited['GBFS1'] = []
for d in data:
    visited['GBFS1'].append(visited_states(*optimized_gbfs(d, h1)))

In [ ]:
visited['GBFS2'] = []
for d in data:
    visited['GBFS2'].append(visited_states(*optimized_gbfs(d, h2)))

In [ ]:
visited['A*1'] = []
for d in data:
    visited['A*1'].append(visited_states(*optimized_a_star(d, h1)))

In [ ]:
visited['A*2'] = []
for d in data:
    visited['A*2'].append(visited_states(*optimized_a_star(d, h2)))

In [ ]:
visited['HCS1'] = []
for d in data:
    visited['HCS1'].append(visited_states(*optimized_hcs(d, h1)))

In [ ]:
visited['HCS2'] = []
for d in data:
    visited['HCS2'].append(visited_states(*optimized_hcs(d, h2)))

In [ ]:
%matplotlib inline
plot_algorithms(visited, 'visited states')